In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import json
from scipy import signal
import plotly.express as px
import plotly.graph_objects as go
import statistics

In [15]:
def extract_PID_data(data, PROTOCOL,LABEL,PLOT_FLAG):

    if (PROTOCOL == 'SAE'):

        if LABEL == 'IMAP':
            PID_TAG = '106'
        elif LABEL == 'ENGINE LOAD':
            PID_TAG = '92'
        elif LABEL == 'ENGINE RPM':
            PID_TAG = '190'
        elif LABEL == 'FUEL RATE':
            PID_TAG = '183'
        elif LABEL == 'MAF':
            PID_TAG = '132'
        elif LABEL == 'BOOST':
            PID_TAG = '102'
        elif LABEL == 'BAROMETER':
            PID_TAG = '108'
        elif LABEL == 'THROTTLE':
            PID_TAG = '91'
        elif LABEL == 'ATGMF': # Eaxuast Gas Flow Rate
            PID_TAG = '3236'
        elif LABEL == 'DPFDP': # DPF Diffrential Pressure (across DPF)
            PID_TAG = '3251'
        elif LABEL == 'SPEED': # Wheep based Vehicle Speed
            PID_TAG = '84'
        elif LABEL == 'DPFINT':# DPF in Temperature Before DPF (DOC out)
            PID_TAG = '3250' #'3250' #4766
        elif LABEL == 'IS': #  regen inhibited
            PID_TAG = '3703'        
        elif LABEL == 'FUEL USE': #  Total fuel used (high precision)
            PID_TAG = '5054'        
        elif LABEL == 'DISTANCE': #  Total distance travelled (high precision)
            PID_TAG = '245'
        elif LABEL == 'SOOTLOAD':
            PID_TAG = '5466' # 245 #3719 generic check 
        elif LABEL == 'EGR':
             PID_TAG = '2659'
        elif LABEL == 'DEF_LEVEL':
             PID_TAG = '1761'
        elif LABEL == 'DEF_DOSING_PRESSURE':
             PID_TAG = '4334'
        elif LABEL == 'DEF_DOSING_RATE':
             PID_TAG = '4331' 
        elif LABEL == 'DEF_MOTOR_RPM':
             PID_TAG = '4374'                    


    elif(PROTOCOL == 'SAE_AVG'):

        if LABEL == 'ENGINE LOAD':
            PID_TAG = 'dff_92_avg'
        elif LABEL == 'ENGINE RPM':
            PID_TAG = 'dff_190_avg'
        elif LABEL == 'THROTTLE':
            PID_TAG = 'dff_91_avg'
        elif LABEL == 'ATGMF': # Eaxuast Gas Flow Rate
            PID_TAG = 'dff_3236_avg'
        elif LABEL == 'DPFDP': # DPF Diffrential Pressure (across DPF)
            PID_TAG = 'dff_3251_avg'
        elif LABEL == 'SPEED': # Wheep based Vehicle Speed
            PID_TAG = 'dff_84_avg'
        elif LABEL == 'DPFINT':# DPF in Temperature Before DPF (DOC out)
            PID_TAG = 'dff_3250_avg' #'3250' #4766
        elif LABEL == 'IS': #  regen inhibited
            PID_TAG = '3703'        
        elif LABEL == 'SOOTLOAD':
            PID_TAG = '5466' # 245 #3719 generic check  

    elif(PROTOCOL == 'ISO'):

        if LABEL == 'IMAP':
            PID_TAG = '0B, 87BC'#MODIFY the "if PID_TAG in State1:" loop (append for loop on top)  
        elif LABEL == 'ENGINE LOAD':
            PID_TAG = '04'
        elif LABEL == 'ENGINE RPM':
            PID_TAG = '0C'
        elif LABEL == 'FUEL RATE':
            PID_TAG = '5E'
        elif LABEL == 'MAF':
            PID_TAG = '10'
        elif LABEL == 'BOOST':
            PID_TAG = '102'
        elif LABEL == 'BAROMETER':
            PID_TAG = '33'
        elif LABEL == 'THROTTLE':
            PID_TAG = '11, 47, 48, 49, 4A, 4B'#MODIFY the "if PID_TAG in State1:" loop (append for loop on top) 
        elif LABEL == 'EGR':
            PID_TAG = '69'
        elif LABEL == 'ATGMF':
            PID_TAG = '9E'         

    #print("PID TAG is-->" + LABEL + " ," +PROTOCOL,"Mapping is --->", PID_TAG)

    Time_vec = []
    Val_vec = []
    #print(len(data[0]))
    #data = data[0]
    #print(len(data))
 
    for data_cnt in range(0,len(data[0])):
        if "pids" in data[0][data_cnt]:
            if len(data[0][data_cnt]['pids'])>0:
                for sub_pid_cnt in range(0,len(data[0][data_cnt]['pids'])):  #this loop
                    State = data[0][data_cnt]['pids'][sub_pid_cnt]
                    #print(State)
                    #print(data_cnt)
                    #for state_cnt in range(0,len(State)):
                    if PID_TAG in State:
                        #print("--------------------------------------------IN----------------------------------")
                        Time_vec.append(State[PID_TAG]['timestamp'])
                        Val_vec.append(State[PID_TAG]['value'])

    #print("Number of time stamp avilables are--->",len(Val_vec))

    if (PLOT_FLAG == 1):            
        plt.title("Time Series") 
        plt.xlabel("Time Stamp") 
        plt.ylabel(LABEL) 
        plt.scatter(Time_vec,Val_vec) 
        plt.show()
                
    return Time_vec,Val_vec

In [12]:
DEF_vehicles_path = "/home/ubuntu/Harleen/papl_vehWithDEF.json"
papl_vehData_path = "/home/ubuntu/vikram/Data/paplAC/withEGR/"
vehicle_ls = []
with open(DEF_vehicles_path) as f:
    DEF_PID_data = json.load(f)
for vehicle_info in DEF_PID_data[1]:
    vehicle_ls.append(vehicle_info[0])
        
        

# DEF Level : 1761
# s

In [10]:
LABELS = ['DEF_LEVEL', 'DEF_DOSING_PRESSURE', 'DEF_DOSING_RATE', 'DEF_MOTOR_RPM']
for vehicle in vehicle_ls:
    obd_data_path = papl_vehData_path + '1696118400000_1711929600000' + "_" + str(vehicle) + "/"
    if os.path.exists(obd_data_path):
        obd_packets_dir = os.listdir(obd_data_path)
        for packet in obd_packets_dir:
            obd_packet_path = obd_data_path + str(packet)
            obd_data = [json.loads(line) for line in  open(obd_packet_path, 'r')]
            for LABEL in LABELS:
                  Time_vec,Val_vec= extract_PID_data(obd_data, 'SAE',LABEL,PLOT_FLAG)
        



['1043935359953534976',
 '906553366081961984',
 '906587728479518720',
 '945305205002469376',
 '906994209133690880',
 '912077595946778624',
 '631520892802871296',
 '1271491538651709440',
 '631530138969833472',
 '1248331825202331648',
 '631530477165027328',
 '1236009076480016384',
 '631530977453146112',
 '1233446254625685504',
 '631554904548179968',
 '631555368278888448',
 '1207271731903856640',
 '631929510001917952',
 '631969644411879424',
 '632306667840602112',
 '632566513798262784',
 '632570636320899072',
 '632857992030621696',
 '632858249363718144',
 '633058752852938752',
 '633058960999518208',
 '634006079201607680',
 '634006596787109888',
 '634343613613998080',
 '635413134051377152',
 '635413403841593344',
 '636523422100226048',
 '636523655962034176',
 '640531462415712256',
 '640531690271285248',
 '640659350389538816',
 '641587040927875072',
 '643174321514086400',
 '649942875752157184',
 '1039226977875132416',
 '696683238281707520',
 '1038841992571781120',
 '1032295106801041408',
 '